In [ ]:
import numpy as np
import pandas as pd
import os
import cv2
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import tensorflow as tf
import keras
from keras import optimizers
from keras.preprocessing.image import load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
from PIL import Image

In [ ]:
file_list = os.listdir('../input/handgesture/build')

images = []
labels = []
for i in range(len(file_list)):
    arr = file_list[i]
    if 'one' in file_list[i]:
        images.append(arr)
        labels.append('1')
    elif 'two' in file_list[i]:
        images.append(arr)
        labels.append('2')
    elif 'three' in file_list[i]:
        images.append(arr)
        labels.append('3')
    elif 'four' in file_list[i]:
        images.append(arr)
        labels.append('4')
    elif 'openHand' in file_list[i]:
        images.append(arr)
        labels.append('5')
    elif 'closeHand' in file_list[i]:
        images.append(arr)
        labels.append('0')
        
df = pd.DataFrame({
    'archivo': images,
    'categoria': labels
})

In [ ]:
# Separamos dos conjuntos, train(80% del dataset) y test(20%).
train, test = train_test_split(df, test_size=0.20, random_state=42)

# Reseteamos los index para vayan al comienzo de daatframe.
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

# Imprimimos el tamano de los dataframes.
print(train.shape, '<-->', test.shape)

In [ ]:
# Generador De Entrenamiento (Preprocesamiento de las imagenes)
TAM_IMG = (100, 100)
"""
Generamos nuevas imagenes en base a las pre existentes, por ejmplo, podemos cambiar el rango de rotacion, 
cortamos la imagen, hacemos zoom, volteamos, entre otras tecnicas para que de esta manera podamos tener
una mayor cantidad de imagenes para entrenar y hacer el testing.

"""
train_image_data = ImageDataGenerator(
    rotation_range = 15,
    rescale = 1./255,
    zoom_range=0.25,
    horizontal_flip = True,
    width_shift_range = 0.1,
    height_shift_range = 0.1
)

# Procedemos ha generar las imagenes y preprocesarlas.
train_generador = train_image_data.flow_from_dataframe(
    train,
    '../input/handgesture/build',
    x_col = 'archivo',
    y_col = 'categoria',
    color_mode="grayscale",
    target_size = TAM_IMG,
    class_mode = 'categorical'
)

# Escalamos la imagen diviendo cada pixel para 255.
test_image_data = ImageDataGenerator(rescale=1./255)
test_generador = test_image_data.flow_from_dataframe(
    test,
    '../input/handgesture/build',
    color_mode="grayscale",
    x_col = 'archivo',
    y_col = 'categoria',
    target_size = TAM_IMG,
    class_mode = 'categorical'
)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


modelo = Sequential()

modelo.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(64, (3, 3), activation='relu'))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))
modelo.add(Dropout(0.25))

modelo.add(Conv2D(128, (3, 3), activation='relu'))
modelo.add(BatchNormalization())
modelo.add(MaxPooling2D(pool_size=(2, 2)))

modelo.add(Flatten())
modelo.add(Dense(256, activation = 'relu'))
modelo.add(BatchNormalization())
modelo.add(Dropout(0.5))
modelo.add(Dense(6, activation='softmax'))

modelo.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [ ]:
train_shape = train.shape[0]
test_shape = test.shape[0]

history = modelo.fit_generator(
    train_generador,
    epochs = 150,
    validation_data = test_generador
)

In [ ]:
def guardar_pesos(nombre_archivo='pesos.h5'):
    modelo.save_weights(nombre_archivo)

def guardar_modelo(nombre_archivo='modelo.json'):
    modelo_json = modelo.to_json()
    with open(nombre_archivo, "w") as json_file:
        json_file.write(modelo_json)
        
guardar_pesos()
guardar_modelo()

In [ ]:
data = '../input/handgesture/build/170openHand.png'

img = Image.open(data).convert('L')
img = img.resize((100, 100))
img = np.asarray(img)
img = img.reshape(-1, 100, 100, 1)
img = img//255
values = modelo.predict(img)
y_classes = values.argmax(axis=-1)
print(y_classes[0])

In [ ]:
import tensorflow as tf

In [ ]:
val = float(tf.__version__.split('.')[0] + '.' + tf.__version__.split('.')[1])
val